In [2]:
%load_ext autoreload
%autoreload 2

from datasets.kaggle import KaggleDataset
import pandas as pd
import numpy as np

np.set_printoptions(precision=4, suppress=True)

In [3]:
dataset = KaggleDataset("Daily")

train_price, _ = dataset.get_price_date_series(
    pd.Timestamp("2014-1-1"), pd.Timestamp("2020-1-1")
)
test_price, _ = dataset.get_price_date_series(
    pd.Timestamp("2020-1-1"), pd.Timestamp("2030-1-1")
)

In [4]:
from algos.hill import HC
from algos.pso import PSO
from algos.woa import WOA

algo = {"hill": HC, "pso": PSO, "woa": WOA}

In [5]:
from bot import sim_bot


def make_eval_fn(signal_fn, data):
    return lambda params: sim_bot(signal_fn(data, *params), data)

In [6]:
from algos.base import BaseAlgo
from signals import complex_bot_signal

complex_eval_fn = make_eval_fn(complex_bot_signal, train_price)


def make_algo_from_HL(algo, h, l):
    return algo(
        complex_eval_fn,
        bounds=[(0, 10)] * 3
        + [h] * 3
        + [(0.01, 1)]
        + [(0, 10)] * 3
        + [l] * 3
        + [(0.01, 1)],
        integer_dims=[3, 4, 5, 10, 11, 12],
        seed=42,
    )


groupA: dict[str, BaseAlgo] = {
    k: make_algo_from_HL(v, (1, 10), (11, 40)) for k, v in algo.items()
}
groupB: dict[str, BaseAlgo] = {
    k: make_algo_from_HL(v, (5, 50), (51, 100)) for k, v in algo.items()
}


In [7]:
# VIS 1

for algo_name, algo_instance in groupA.items():
    algo_instance.optimise()
    algo_instance.plot(title=algo_name)

In [8]:
# VIS 2

# optimise group B

for algo_name, algo_instance in groupB.items():
    algo_instance.optimise()

import pandas as pd

# build a dataframe with the algo name, its best score, and its group
group_A_df = pd.DataFrame(
    {
        "algo": [algo_name for algo_name in groupA.keys()],
        "score": [algo.best_fitness for algo in groupA.values()],
        "group": ["A"] * len(groupA),
        "type": ["complex"] * len(groupA),
    }
)
group_B_df = pd.DataFrame(
    {
        "algo": [algo_name for algo_name in groupB.keys()],
        "score": [algo.best_fitness for algo in groupB.values()],
        "group": ["B"] * len(groupB),
        "type": ["complex"] * len(groupB),
    }
)
# combine the two dataframes
combined_df = pd.concat([group_A_df, group_B_df], ignore_index=True)

import plotly.express as px

px.bar(combined_df, x="algo", y="score", color="group", barmode="group").show()


In [9]:
# VIS 3

from signals import sma2_bot_signal, smaema_bot_signal

def make_sma2_algo_from_HL(algo, h, l):
    return algo(make_eval_fn(sma2_bot_signal, train_price), bounds=[h, l], seed=42)


def make_smaema_algo_from_HL(algo, h, l):
    return algo(
        make_eval_fn(smaema_bot_signal, train_price), bounds=[h, l, (0.01, 1)], seed=42
    )



groupB_sma2: dict[str, BaseAlgo] = {
    k: make_sma2_algo_from_HL(v, (5, 50), (51, 100)) for k, v in algo.items()
}
groupB_smaema: dict[str, BaseAlgo] = {
    k: make_smaema_algo_from_HL(v, (5, 50), (51, 100)) for k, v in algo.items()
}

# optimise group A and B for sma2 and smaema
for algo_name, algo_instance in groupB_sma2.items():
    algo_instance.optimise()
for algo_name, algo_instance in groupB_smaema.items():
    algo_instance.optimise()

# build a dataframe that contains algo name, best fitness, group, and type (sma2, smaema, complex)
group_B_sma2_df = pd.DataFrame(
    {
        "algo": [algo_name for algo_name in groupB_sma2.keys()],
        "score": [algo.best_fitness for algo in groupB_sma2.values()],
        "group": ["B"] * len(groupB_sma2),
        "type": ["sma2"] * len(groupB_sma2),
    }
)
group_B_smaema_df = pd.DataFrame(
    {
        "algo": [algo_name for algo_name in groupB_smaema.keys()],
        "score": [algo.best_fitness for algo in groupB_smaema.values()],
        "group": ["B"] * len(groupB_smaema),
        "type": ["smaema"] * len(groupB_smaema),
    }
)

# combine the dataframes
bigger_combined_df = pd.concat(
    [
        group_B_sma2_df,
        group_B_smaema_df,
        group_B_df,
    ],
    ignore_index=True,
)

fig = px.bar(bigger_combined_df, x="algo", y="score", color="type", barmode="group")
fig.update_traces(texttemplate='%{y:.2f}', textposition='outside')
fig.show()
